In [3]:
import datetime
from txgnn import TxData, TxGNN, TxEval
import builtins
import time
import pandas as pd
import argparse
import wandb

In [2]:
from importlib import reload
import txgnn
reload(txgnn)
from txgnn import TxData, TxGNN, TxEval

In [3]:
# LOAD DATA

def get_timestamp():
    return datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")

data_path = '/om/user/tysinger/kg/'

print(f'[{get_timestamp()}] Loading data ...')
TxData_inst = TxData(data_folder_path = data_path)
TxData_inst.prepare_split(split = 'random', seed = 42, no_kg = False)
print(f'[{get_timestamp()}] Loaded data!')
print(f'[{get_timestamp()}] Initializing model ...')

[2024-04-25 19:06:07] Loading data ...
Found local copy...
Found local copy...
Found local copy...
Found saved processed KG... Loading...
Splits detected... Loading splits....
Creating DGL graph....
Done!
[2024-04-25 19:06:30] Loaded data!
[2024-04-25 19:06:30] Initializing model ...


In [4]:
load_name = 'protgnn_pretrain_noesm'
esm = False
TxGNN_model = TxGNN(data = TxData_inst, 
                        weight_bias_track = True,
                        proj_name = 'MEng',
                        exp_name = f'Finetune ProtGNN No ESM',
                        device = 'cuda:0'
                    )
TxGNN_model.load_pretrained(f'/om/user/tysinger/models/{load_name}', esm=esm)
etype = 'MF/BP'

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: eptysinger. Use `wandb login --relogin` to force relogin


{'n_hid': 512, 'n_inp': 512, 'n_out': 1024, 'proto': False, 'proto_num': 3, 'attention': False, 'sim_measure': 'all_nodes_profile', 'bert_measure': 'disease_name', 'agg_measure': 'rarity', 'num_walks': 200, 'walk_mode': 'bit', 'path_length': 2, 'esm': False}


In [5]:
TxGNN_model.finetune(save_path = '/om/user/tysinger/models', 
                    name = 'finetune_debug',
                    n_epoch = 150, 
                    learning_rate = 1e-3,
                    train_print_per_n = 5,
                    valid_per_n = 10,
                    save_per_n = 1000,
                    b=None,
                    edge_type = etype,
                    sweep_wandb=None)
print(f'[{get_timestamp()}] Finetune done! ')
wandb.finish()

Epoch: 0 LR: 0.00100 Loss 0.7230, Train Micro AUROC 0.6010 Train Micro AUPRC 0.5691 Train Macro AUROC 0.5958 Train Macro AUPRC 0.5761
----- AUROC Performance in Each Relation -----
('gene/protein', 'molfunc_protein', 'molecular_function'): 0.6724953072640718
('molecular_function', 'rev_molfunc_protein', 'gene/protein'): 0.4899269720383377
('gene/protein', 'bioprocess_protein', 'biological_process'): 0.7069708776191928
('biological_process', 'rev_bioprocess_protein', 'gene/protein'): 0.513614890243263
----- AUPRC Performance in Each Relation -----
('gene/protein', 'molfunc_protein', 'molecular_function'): 0.6185947862185464
('molecular_function', 'rev_molfunc_protein', 'gene/protein'): 0.49329407903774913
('gene/protein', 'bioprocess_protein', 'biological_process'): 0.6752081234821062
('biological_process', 'rev_bioprocess_protein', 'gene/protein'): 0.5173860253750316
----------------------------------------------
Validation.....
Epoch: 0 LR: 0.00100 Validation Loss 0.7172,  Validation 

wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


LR,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Testing Loss,▁
Testing Macro AUPRC,▁
Testing Macro AUROC,▁
Testing Micro AUPRC,▁
Testing Micro AUROC,▁
Training Loss,█▇▆▅▄▄▃▃▃▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Training Macro AUPRC,▁▅▆▆▇▇▇▇██████████████████████
Training Macro AUROC,▁▅▅▆▇▇▇▇▇█████████████████████
Training Micro AUPRC,▁▅▆▆▇▇▇▇██████████████████████
Training Micro AUROC,▁▅▆▆▆▇▇▇▇█████████████████████


In [6]:
TxGNN_model.save_model('/om/user/tysinger/models/protgnn_finetuned_noesm')
TxGNN_model.retrieve_embedding(path = '/om/user/tysinger/embeddings', save_name='protgnn_finetuned_noesm')

Retrieving embeddings ...


{'anatomy': tensor([[-1.9802, -0.5453,  0.5164,  ...,  0.6677, -1.7421, -5.2730],
         [ 0.0630, -0.0508,  0.0190,  ..., -0.3061,  0.0407, -0.0413],
         [ 0.2067, -0.0705, -0.0282,  ..., -0.1096, -0.0122, -0.1034],
         ...,
         [ 0.1044,  0.0409,  0.0238,  ..., -0.2886, -0.2127, -0.1201],
         [ 0.1044,  0.0409,  0.0238,  ..., -0.2886, -0.2127, -0.1201],
         [ 0.1044,  0.0409,  0.0238,  ..., -0.2886, -0.2127, -0.1201]]),
 'biological_process': tensor([[ 0.2110, -0.0167, -0.3080,  ...,  0.1197, -0.0977, -0.1972],
         [ 3.5733, -0.1231,  0.2335,  ...,  1.4229, -1.5625,  0.4836],
         [ 0.7233,  0.5643, -1.2228,  ..., -0.9294, -0.3532,  1.6331],
         ...,
         [ 0.0751,  0.0390, -0.1800,  ...,  0.1959, -0.0188, -0.2062],
         [ 0.1788,  0.2016,  0.0113,  ...,  0.0774, -0.1147,  0.0995],
         [ 0.0909,  0.1762, -0.5710,  ...,  0.1129, -0.1236, -0.3181]]),
 'cellular_component': tensor([[-0.2363, -0.4105, -0.0920,  ..., -0.3093,  0.0604, 

## Node Type Ablation Testing

In [1]:
from importlib import reload
import txgnn
reload(txgnn)
from txgnn import TxData, TxGNN, TxEval
from txgnn import utils

/home/tysinger/.conda/envs/txgnn_env2/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
def get_timestamp():
    return datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")

data_path = '/om/user/tysinger/kg/'

print(f'[{get_timestamp()}] Loading data ...')
TxData_inst = TxData(data_folder_path = data_path)
TxData_inst.prepare_split(split = 'random', seed = 42, no_kg = False, remove_node_type='anatomy')

[2024-05-02 23:08:23] Loading data ...
Found local copy...
Found local copy...
Found local copy...
Found saved processed KG... Loading...
Splits detected... Loading splits....
Removing node type: anatomy
Creating DGL graph....
Done!


In [8]:
node_types = ['anatomy', 'biological_process', 'cellular_component', 'disease', 'drug', 'effect/phenotype', 'exposure', 'molecular_function', 'pathway']

In [9]:
for node_type in node_types:
    print(node_type.split('/')[0])

anatomy
biological_process
cellular_component
disease
drug
effect
exposure
molecular_function
pathway


In [23]:
TxData_inst.G

Graph(num_nodes={'anatomy': 14033, 'biological_process': 28642, 'cellular_component': 4176, 'disease': 17080, 'drug': 7957, 'effect/phenotype': 15311, 'exposure': 818, 'gene/protein': 27610, 'molecular_function': 11169, 'pathway': 2516},
      num_edges={('anatomy', 'anatomy_anatomy', 'anatomy'): 23328, ('anatomy', 'rev_anatomy_protein_absent', 'gene/protein'): 16531, ('anatomy', 'rev_anatomy_protein_present', 'gene/protein'): 1262006, ('biological_process', 'bioprocess_bioprocess', 'biological_process'): 87924, ('biological_process', 'rev_bioprocess_protein', 'gene/protein'): 120369, ('biological_process', 'rev_exposure_bioprocess', 'exposure'): 1351, ('cellular_component', 'cellcomp_cellcomp', 'cellular_component'): 8056, ('cellular_component', 'rev_cellcomp_protein', 'gene/protein'): 69328, ('cellular_component', 'rev_exposure_cellcomp', 'exposure'): 9, ('disease', 'disease_disease', 'disease'): 53522, ('disease', 'disease_phenotype_negative', 'effect/phenotype'): 991, ('disease', '

In [7]:
TxData_inst.G

Graph(num_nodes={'biological_process': 28642, 'cellular_component': 4176, 'disease': 17080, 'drug': 7957, 'effect/phenotype': 15311, 'exposure': 818, 'gene/protein': 27610, 'molecular_function': 11169, 'pathway': 2516},
      num_edges={('biological_process', 'bioprocess_bioprocess', 'biological_process'): 87924, ('biological_process', 'rev_bioprocess_protein', 'gene/protein'): 120369, ('biological_process', 'rev_exposure_bioprocess', 'exposure'): 1351, ('cellular_component', 'cellcomp_cellcomp', 'cellular_component'): 8056, ('cellular_component', 'rev_cellcomp_protein', 'gene/protein'): 69328, ('cellular_component', 'rev_exposure_cellcomp', 'exposure'): 9, ('disease', 'disease_disease', 'disease'): 53522, ('disease', 'disease_phenotype_negative', 'effect/phenotype'): 991, ('disease', 'disease_phenotype_positive', 'effect/phenotype'): 124951, ('disease', 'rev_contraindication', 'drug'): 25498, ('disease', 'rev_disease_protein', 'gene/protein'): 66841, ('disease', 'rev_exposure_disease'

In [20]:
g

Graph(num_nodes={'anatomy': 14033, 'biological_process': 28642, 'cellular_component': 4176, 'drug': 7957, 'effect/phenotype': 15311, 'exposure': 818, 'gene/protein': 27610, 'molecular_function': 11169, 'pathway': 2516},
      num_edges={('anatomy', 'anatomy_anatomy', 'anatomy'): 23328, ('anatomy', 'rev_anatomy_protein_absent', 'gene/protein'): 16531, ('anatomy', 'rev_anatomy_protein_present', 'gene/protein'): 1262006, ('biological_process', 'bioprocess_bioprocess', 'biological_process'): 87924, ('biological_process', 'rev_bioprocess_protein', 'gene/protein'): 120369, ('biological_process', 'rev_exposure_bioprocess', 'exposure'): 1351, ('cellular_component', 'cellcomp_cellcomp', 'cellular_component'): 8056, ('cellular_component', 'rev_cellcomp_protein', 'gene/protein'): 69328, ('cellular_component', 'rev_exposure_cellcomp', 'exposure'): 9, ('drug', 'drug_drug', 'drug'): 2221622, ('drug', 'drug_effect', 'effect/phenotype'): 53852, ('drug', 'drug_protein', 'gene/protein'): 21324, ('effect